In [1]:
import numpy as np # linear algebra
import pandas as pd
import sqlite3

In [2]:
df_olist_customers = pd.read_csv('olist_customers_dataset.csv')
df_olist_sellers = pd.read_csv('olist_sellers_dataset.csv')
df_olist_order_reviews= pd.read_csv('olist_order_reviews_dataset.csv')
df_olist_order_items= pd.read_csv('olist_order_items_dataset.csv')
df_olist_products= pd.read_csv('olist_products_dataset.csv')
df_olist_geolocation= pd.read_csv('olist_geolocation_dataset.csv')
df_product_category_name_translation= pd.read_csv('product_category_name_translation.csv')
df_olist_orders = pd.read_csv('olist_orders_dataset.csv')
df_olist_order_payments= pd.read_csv('olist_order_payments_dataset.csv')


In [4]:
df_olist_customers.head()

from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)


In [5]:
# export the dataframe as a table 'playstore' to the sqlite engine
df_olist_customers.to_sql("olist_customers", con =engine)
df_olist_sellers.to_sql("olist_sellers", con =engine)
df_olist_order_reviews.to_sql("olist_order_reviews", con =engine)
df_olist_order_items.to_sql("olist_order_items", con =engine)
df_olist_products.to_sql("olist_products_dataset", con =engine)
df_olist_geolocation.to_sql("olist_geolocation", con =engine)
df_product_category_name_translation.to_sql("product_category_name_translation", con =engine)
df_olist_orders.to_sql("olist_orders", con =engine)
df_olist_order_payments.to_sql("olist_order_payments", con =engine)
df_olist_order_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [6]:
sql='''

Select * from olist_customers
limit 5


''';


df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [7]:
query = """
WITH jan_2018_orders AS (
    SELECT o.order_id, r.review_score
    FROM olist_orders o
    LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
    WHERE strftime('%Y-%m', o.order_purchase_timestamp) = '2018-01'
)
SELECT
    COUNT(CASE WHEN review_score = 5 THEN 1 END) AS five_star_orders,
    COUNT(*) AS total_orders,
    ROUND(100.0 * COUNT(CASE WHEN review_score = 5 THEN 1 END) / COUNT(*), 2) AS percentage_five_star
FROM jan_2018_orders;
"""

result = pd.read_sql_query(query, con=engine)
print(result)

   five_star_orders  total_orders  percentage_five_star
0              4097          7313                 56.02


In [8]:
import pandas as pd

query = """
SELECT
  strftime('%Y', order_purchase_timestamp) AS year,
  COUNT(order_id) AS total_orders
FROM olist_orders
GROUP BY year
ORDER BY year;
"""

df_yearly_trend = pd.read_sql_query(query, con=engine)
print(df_yearly_trend)

   year  total_orders
0  2016           329
1  2017         45101
2  2018         54011


In [9]:
query = """
WITH order_values AS (
  SELECT
    o.customer_id,
    p.order_id,
    SUM(p.payment_value) AS order_total
  FROM olist_orders o
  JOIN olist_order_payments p ON o.order_id = p.order_id
  GROUP BY p.order_id
)

SELECT
  customer_id,
  ROUND(AVG(order_total), 2) AS avg_order_value
FROM order_values
GROUP BY customer_id
ORDER BY avg_order_value DESC;
"""

df_avg_order_value = pd.read_sql_query(query, con=engine)
df_avg_order_value.head()

,customer_id,avg_order_value
0,1617b1357756262bfa56ab541c47bc16,13664.08
1,ec5b2ba62e574342386871631fafd3fc,7274.88
2,c6e2731c5b391845f6800c97401a43a9,6929.31
3,f48d464a0baaea338cb25f816991ab1f,6922.21
4,3fd6777bbce08a352fddd04e4a7cc8f6,6726.66


In [10]:
query = """
SELECT
  c.customer_city,
  ROUND(SUM(p.payment_value), 2) AS total_revenue
FROM olist_orders o
JOIN olist_order_payments p ON o.order_id = p.order_id
JOIN olist_customers c ON o.customer_id = c.customer_id
WHERE strftime('%Y', o.order_purchase_timestamp) BETWEEN '2016' AND '2018'
GROUP BY c.customer_city
ORDER BY total_revenue DESC
LIMIT 5;
"""

df_top_cities_revenue = pd.read_sql_query(query, con=engine)
df_top_cities_revenue

,customer_city,total_revenue
0,sao paulo,2203373.09
1,rio de janeiro,1161927.36
2,belo horizonte,421765.12
3,brasilia,354216.78
4,curitiba,247392.48


In [11]:
query = """
SELECT
  c.customer_state,
  ROUND(SUM(p.payment_value), 2) AS total_revenue
FROM olist_orders o
JOIN olist_order_payments p ON o.order_id = p.order_id
JOIN olist_customers c ON o.customer_id = c.customer_id
WHERE strftime('%Y', o.order_purchase_timestamp) BETWEEN '2016' AND '2018'
GROUP BY c.customer_state
ORDER BY total_revenue DESC;
"""

df_state_revenue = pd.read_sql_query(query, con=engine)
df_state_revenue.head()

,customer_state,total_revenue
0,SP,5998226.96
1,RJ,2144379.69
2,MG,1872257.26
3,RS,890898.54
4,PR,811156.38


In [12]:
query = """
WITH seller_stats AS (
  SELECT
    s.seller_id,
    COUNT(oi.order_item_id) AS total_items_sold,
    ROUND(SUM(oi.price + oi.freight_value), 2) AS total_revenue,
    COUNT(DISTINCT o.customer_id) AS unique_customers
  FROM olist_order_items oi
  JOIN olist_orders o ON oi.order_id = o.order_id
  JOIN olist_sellers s ON oi.seller_id = s.seller_id
  GROUP BY s.seller_id
),
five_star_reviews AS (
  SELECT
    oi.seller_id,
    COUNT(r.review_score) AS five_star_count
  FROM olist_order_items oi
  JOIN olist_order_reviews r ON oi.order_id = r.order_id
  WHERE r.review_score = 5
  GROUP BY oi.seller_id
)

SELECT
  ss.seller_id,
  ss.total_items_sold,
  ss.total_revenue,
  ss.unique_customers,
  COALESCE(fr.five_star_count, 0) AS five_star_reviews
FROM seller_stats ss
LEFT JOIN five_star_reviews fr ON ss.seller_id = fr.seller_id
ORDER BY
  ss.total_revenue DESC,
  ss.total_items_sold DESC,
  ss.unique_customers DESC,
  five_star_reviews DESC
LIMIT 10;
"""

df_top_sellers = pd.read_sql_query(query, con=engine)
df_top_sellers.head()

,seller_id,total_items_sold,total_revenue,unique_customers,five_star_reviews
0,4869f7a5dfa277a7dca6462dcf3b52b2,1156,249640.70,1132,683
1,7c67e1448b00f6e969d365cea6b010ab,1364,239536.44,982,437
2,53243585a1d6dc2643021fd1853d8905,410,235856.68,358,210
3,4a3ca9315b744ce9f8e9374361493884,1987,235539.96,1806,947
4,fa1c13f2614d7b5c4749cbc52fecda94,586,204084.73,585,394


In [13]:
query = """
SELECT
  c.customer_state,
  COUNT(CASE WHEN o.order_status = 'delivered' THEN 1 END) * 1.0 / COUNT(*) AS delivery_success_rate,
  COUNT(*) AS total_orders,
  COUNT(CASE WHEN o.order_status = 'delivered' THEN 1 END) AS delivered_orders
FROM olist_orders o
JOIN olist_customers c ON o.customer_id = c.customer_id
GROUP BY c.customer_state
ORDER BY delivery_success_rate DESC;
"""

df_delivery_rate = pd.read_sql_query(query, con=engine)
df_delivery_rate.head()

,customer_state,delivery_success_rate,total_orders,delivered_orders
0,AC,0.987654,81,80
1,AP,0.985294,68,67
2,ES,0.981308,2033,1995
3,MS,0.980420,715,701
4,AM,0.979730,148,145


In [14]:
query = """
WITH payment_by_category AS (
  SELECT
    p.payment_type,
    pr.product_category_name,
    COUNT(*) AS payment_count
  FROM olist_order_payments p
  JOIN olist_order_items oi ON p.order_id = oi.order_id
  JOIN olist_products_dataset pr ON oi.product_id = pr.product_id
  GROUP BY pr.product_category_name, p.payment_type
),
ranked_payment AS (
  SELECT *,
    RANK() OVER (PARTITION BY product_category_name ORDER BY payment_count DESC) AS rnk
  FROM payment_by_category
)
SELECT
  product_category_name,
  payment_type,
  payment_count
FROM ranked_payment
WHERE rnk = 1
ORDER BY payment_count DESC;
"""

df_payment_pref = pd.read_sql_query(query, con=engine)
df_payment_pref.head()

,product_category_name,payment_type,payment_count
0,cama_mesa_banho,credit_card,8959
1,beleza_saude,credit_card,7566
2,esporte_lazer,credit_card,6635
3,moveis_decoracao,credit_card,6379
4,informatica_acessorios,credit_card,5436
